In [1]:
import itertools
import numpy as np
import os
import seaborn as sns
from tqdm import tqdm
from dataclasses import asdict, dataclass, field
import vsketch
import shapely.geometry as sg
from shapely.geometry import box, MultiLineString, Point, MultiPoint, Polygon, MultiPolygon, LineString
import shapely.affinity as sa
import shapely.ops as so
import matplotlib.pyplot as plt
import pandas as pd

import vpype_cli
from typing import List, Generic
from genpen import genpen as gp, utils as utils
from scipy import stats as ss
import geopandas
from shapely.errors import TopologicalError
import functools
%load_ext autoreload
%autoreload 2
import vpype
from skimage import io
from pathlib import Path
import fn
import bezier

from sklearn.preprocessing import minmax_scale
from skimage import feature
from genpen.utils import Paper
from genpen.genpen import *
from genpen import subdivide as sd
from functools import partial
from genpen.grower import Grower, GrowerParams

In [3]:
# make page
paper_size = 'A2'
border:float=45
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
drawbox.bounds

In [ ]:
drawbox

In [ ]:
split_func = functools.partial(sd.split_random_bezier, x0=0.2, x1=0.75, n_eval_points=50)

xgen = ss.uniform(loc=0.4, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)

# x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
# x1gen = ss.uniform(loc=0.65, scale=0.01).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)

In [ ]:
target = Point(140, 325)
target = drawbox.centroid

In [ ]:
dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(0.99, 0.3,), d_range=(0, 200))
cp = sd.ContinuePolicy(dist_from_center)
polys = sd.very_flex_rule_recursive_split(poly=drawbox, split_func=split_func, continue_func=cp, depth_limit=14, buffer_kwargs={'distance':1e-6})


bps = gp.merge_Polygons(polys)

In [ ]:

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:
layers = []

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.005, 0.01)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.22),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.1, 0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

# more interesting initialization

In [ ]:
# make page
paper_size = 'A2'
border:float=45
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
params = GrowerParams(
    rad_func='loss_scaled_rad',
    rad_range=(70, 60),
    loss_range=(40, 100),
    n_pts_eval_per_iter=55,
    n_pts_add_per_iter=1,
    pt_to_poly_func='buffer_pt'
)
g = Grower(poly=drawbox.centroid.buffer(0.1), params=params)

In [ ]:
g.grow(6)

In [ ]:
# polys = g.mpoly.buffer(-5)
# polys = g.mpoly

polys = gp.make_like(g.mpoly, drawbox)

In [ ]:
bg = drawbox.difference(polys)

In [ ]:
polys = list(polys)
# polys.append(bg)

In [ ]:
# split_func = functools.partial(sd.split_random_bezier, x0=0.2, x1=0.75, n_eval_points=50)

xgen = ss.uniform(loc=0.5, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)

# x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
# x1gen = ss.uniform(loc=0.65, scale=0.01).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)

In [ ]:
target = Point(140, 325)
# target = drawbox.centroid

In [ ]:
xgen = ss.uniform(loc=0.4, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)
split_polys = []
for poly in polys:
    dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(0.99, 0.3,), d_range=(0, 200))
    cp = sd.ContinuePolicy(dist_from_center)
    sps = sd.very_flex_rule_recursive_split(poly=poly, split_func=split_func, continue_func=cp, depth_limit=2, buffer_kwargs={'distance':1e-6})
    split_polys.append(sps)
    
xgen = ss.uniform(loc=0.5, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)
dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(0.99, 0.3,), d_range=(0, 200))
cp = sd.ContinuePolicy(dist_from_center)
sps = sd.very_flex_rule_recursive_split(poly=bg, split_func=split_func, continue_func=cp, depth_limit=5, buffer_kwargs={'distance':1e-6})
split_polys.append(sps)


bps = gp.merge_Polygons(split_polys)

In [ ]:

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:
layers = []

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.001, 0.005)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.21),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.1, 0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

# more interesting initialization

In [ ]:
# make page
paper_size = '6x6 inches'
border:float=15
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [ ]:
params = GrowerParams(
    rad_func='loss_scaled_rad',
    rad_range=(20, 10),
    loss_range=(20, 100),
    n_pts_eval_per_iter=55,
    n_pts_add_per_iter=1,
    pt_to_poly_func='reg_poly'
)
g = Grower(poly=drawbox.centroid.buffer(40), params=params)

In [ ]:
g.grow(2)

In [ ]:
# polys = g.mpoly.buffer(-1)
polys = g.mpoly

# polys = gp.make_like(g.mpoly, drawbox)

In [ ]:
bg = drawbox.difference(g.agg_poly)

In [ ]:
polys = gp.merge_Polygons(polys)
# polys.append(bg)

In [ ]:
# split_func = functools.partial(sd.split_random_bezier, x0=0.2, x1=0.75, n_eval_points=50)

xgen = ss.uniform(loc=0.5, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)

# x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
# x1gen = ss.uniform(loc=0.65, scale=0.01).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)

In [ ]:
# target = Point(140, 325)
target = drawbox.centroid

In [ ]:
split_polys =[]
    
x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
x1gen = ss.uniform(loc=0.65, scale=0.1).rvs
split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)
dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(1, 0.3,), d_range=(0, 20))
cp = sd.ContinuePolicy(dist_from_center)

sps = sd.very_flex_rule_recursive_split(poly=drawbox, split_func=split_func, continue_func=cp, depth_limit=2, buffer_kwargs={'distance':1e-6})
sps = gp.merge_Polygons(sps)
split_polys.append(sps)


bps = gp.merge_Polygons(split_polys)

In [ ]:

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

In [ ]:
n_layers = 1

In [ ]:
layers = []

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.001, 0.005)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.21),
            degrees=np.random.uniform(40,60),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.001, 0.005)).rvs
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.21),
            degrees=np.random.uniform(20,40),
            poly_to_fill=p, 
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [ ]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.3mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.1, 0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

In [ ]:
plot_id = fn.new_plot_id()

In [ ]:
savedir='/home/naka/art/plotter_svgs'

In [ ]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

# remove

In [2]:
# make page
paper_size = '5x7 inches'
border:float=10
paper = Paper(paper_size)

drawbox = paper.get_drawbox(border)

In [3]:
poly = drawbox

In [4]:
# split_func = functools.partial(sd.split_random_bezier, x0=0.2, x1=0.75, n_eval_points=50)

xgen = ss.uniform(loc=0.5, scale=0.01).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)

# x0gen = ss.uniform(loc=0.15, scale=0.01).rvs
# x1gen = ss.uniform(loc=0.65, scale=0.01).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)

In [5]:
# target = Point(140, 325)
target = drawbox.centroid

In [6]:
drawbox.bounds

(10.0, 10.0, 117.0, 167.79999999999998)

In [7]:
split_polys =[]
    
x0gen = ss.uniform(loc=0.25, scale=0.01).rvs
x1gen = ss.uniform(loc=0.75, scale=0.1).rvs
# split_func = functools.partial(sd.split_random_line_gen, x0gen=x0gen, x1gen=x1gen)
xgen = ss.uniform(loc=0.6, scale=0.001).rvs
split_func = functools.partial(sd.split_along_longest_side_of_min_rectangle, xgen=xgen)
dist_from_center = partial(sd.distance_from_pt, target=target, p_range=(1, 0.1,), d_range=(0, 60))
cp = sd.ContinuePolicy(dist_from_center)

sps = sd.very_flex_rule_recursive_split(poly=poly, split_func=split_func, continue_func=cp, depth_limit=5, buffer_kwargs={'distance':1e-6})
sps = gp.merge_Polygons(sps)
split_polys.append(sps)


bps = gp.merge_Polygons(split_polys)

In [8]:
bps = gp.make_like(bps, drawbox)

In [9]:

   
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.5mm')
sk.geometry(bps.boundary)

# tolerance=0.5

sk.display()

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-09-22T22:29:23.093859 
 
 
 
 
 
 <polygon points="262.0421,126.3608 262.0421,126.3608 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 280.7623,37.7953 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608"/>
 <polygon points="370.2008,37.7953 370.2008,37.7953 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.2019,143.722 339.202,143.722 339.202,143.722 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,166.8981 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 442.2047,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953 370.2008,37.7953"/>
 <polygon points="249.156,187.3254 249.156,187.3254 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 327.055,213.7344 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 381.1995,153.1717 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 262.0421,126.3608 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.156,187.3254 249.

In [428]:
n_layers = 1

In [451]:
layers = []

In [452]:

for ii in range(n_layers):
    fills = []
    for p in bps:
        xjitter_func = 0
        yjitter_func = ss.norm(loc=0, scale=np.random.uniform(0.001, 0.005)).rvs
        d = p.distance(drawbox.centroid)
        angle= (np.interp(d, (0, 160), (0, 360)) // 30) * 30
        bhf = BezierHatchFill(
            spacing=np.random.uniform(0.2, 0.21),
            degrees=angle,
            poly_to_fill=p,
            xjitter_func=xjitter_func, 
            yjitter_func=yjitter_func,
            fill_inscribe_buffer=1.4,
            n_nodes_per_line=5,
            n_eval_points=6,
        )
        fills.append(bhf.p)

    fills = [f for f in fills if f.length > 0]
    layer = gp.merge_LineStrings(fills)
    layers.append(layer)

In [453]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.08mm')
for i, layer in enumerate(layers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.05, 0.1, 0.2,  0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
sk.vpype('linesimplify --tolerance 0.1 linesort')

# sk.display(color_mode='layer')

In [454]:
def vsketch_to_shapely(sketch):
    return [[LineString([Point(pt.real, pt.imag) for pt in lc]) for lc in layer] for layer in sketch.document.layers.values()]

In [455]:
layer = sk.document.layers[1]

In [456]:
mls = gp.make_like(MultiLineString([LineString([Point(pt.real, pt.imag) for pt in lc]) for lc in layer]), drawbox)

In [457]:
ds = [ls.distance(drawbox.centroid) for ls in mls]

In [458]:
ds = np.array(ds) ** 0.5
ds = ds/ ds.sum()

In [459]:
frac_keep = 0.85
n_keep = int(frac_keep * len(mls))
pmls = MultiLineString(list(np.random.choice(mls, size=n_keep, replace=False, p=ds)))
rlayers = [pmls]

<ipython-input-459-6edad6b45b36>:3: FutureWarning: The input object of type 'LineString' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'LineString', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  pmls = MultiLineString(list(np.random.choice(mls, size=n_keep, replace=False, p=ds)))
<ipython-input-459-6edad6b45b36>:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  pmls = MultiLineString(list(np.random.choice(mls, size=n_keep, replace=False, p=ds)))


In [460]:
sk = vsketch.Vsketch()
sk.size(paper.page_format_mm)
sk.scale('1mm')
sk.penWidth('0.08mm')
for i, layer in enumerate(rlayers):
    sk.stroke(i+1)
    sk.geometry(layer)

for tolerance in [0.05, 0.1, 0.2,  0.3, 0.5, 0.7]:
    sk.vpype(f'linemerge --tolerance {tolerance}mm')
    sk.vpype('linesimplify --tolerance 0.1 linesort')

sk.display(color_mode='layer')

<?xml version="1.0" encoding="utf-8" ?>
 
 
 
 
 image/svg+xml 
 
 2021-09-19T22:27:38.606652 
 
 
 
 
 
 
 
 
 <polyline points="414.7391,160.6412 442.2047,113.0661 442.2047,111.5103 413.9426,160.463 414.2023,160.5211 442.2047,209.0257 442.2047,210.572 413.1506,160.2859 442.2047,109.9619 442.2047,108.4158 412.3551,160.108 412.1442,160.0608 442.2047,212.1367 442.2047,213.7187 411.1004,159.8274 410.7498,159.749 442.2047,105.2797 442.2047,103.7123 409.9474,159.5695 442.2047,215.2644 442.2047,216.8251 409.0369,159.3659 409.1458,159.3902 442.2047,102.151 442.2047,100.6087 408.3519,159.2127 408.0229,159.1391 442.2047,218.3586"/>
 
 
 
 
 
 
 
 <polyline points="382.8529,153.5097 442.2047,50.7033 442.2047,52.2809 383.6512,153.6883 383.3369,153.618 440.2564,252.0955 439.1339,251.7138 382.3138,153.3892 382.0528,153.3308 442.2047,49.1177 442.2047,47.5493 381.2488,153.151 438.0175,251.3342 442.2047,253.7151 442.2047,254.6083 434.372,250.0947 434.6664,250.1948 378.2159,152.4727 378.0403,152.4334 442.2047,41.3263 442.2047,42.8757 378.8478,152.614 379.2512,152.7042 435.7836,250.5746 436.8928,250.9518 380.2642,152.9308 380.4488,152.9721 386.1042,143.2091 442.2047,45.9905 442.2047,44.4254 379.6488,152.7931"/>
 
 
 <polyline points="368.4942,150.2984 433.4885,37.7953 434.3798,37.7953 369.2923,150.4769 369.0204,150.416 424.7509,246.8233 423.653,246.45 368.0036,150.1886 367.6959,150.1198 432.5949,37.7953 431.6958,37.7953 366.888,149.9391 422.5314,246.0687 423.0648,246.25 442.2047,257.2795 442.2047,258.1749 419.1783,244.9285 363.8648,149.263 363.7086,149.2281 428.094,37.7953 428.9835,37.7953 364.4985,149.4047 364.9004,149.4946 420.2962,245.3086 421.4099,245.6873 365.9294,149.7247 366.0905,149.7608 430.7885,37.7953 429.878,37.7953 365.2906,149.5819"/>
 
 
 <polyline points="393.615,236.2365 340.3004,143.9927 340.5821,144.0557 402.0388,37.7953 402.9313,37.7953 341.3305,144.2231 394.7301,236.6156 395.8483,236.9958 342.3628,144.454 342.1755,144.4121 403.8134,37.7953 404.7276,37.7953 342.9761,144.5911 343.3935,144.6845 396.962,237.3746 396.7795,237.3125 442.2047,263.49 442.2047,262.6171 400.3151,238.5147 346.4909,145.3772 346.1387,145.2985 408.3074,37.7953 407.4233,37.7953 345.3615,145.1246 399.2098,238.1388 398.0821,237.7554 344.4266,144.9156 344.5675,144.9471 406.5341,37.7953 405.634,37.7953 343.7699,144.7687"/>
 
 
 <polyline points="332.6057,142.2718 393.0494,37.7953 392.1382,37.7953 331.7994,142.0914 332.0724,142.1525 384.6889,233.2014 385.0534,233.3254 262.8638,233.2083 239.3992,233.1522 239.4298,233.0081 200.5705,232.9545 119.6982,232.9607 119.5617,233.1657 119.6112,148.0025 119.4349,147.9101 119.4277,37.7953 118.6598,37.7953 118.6585,147.5033 118.8396,147.5982 118.7896,234.3252 118.6513,234.5329 239.102,234.5548 239.0729,234.6925 320.5133,234.8369 387.6717,234.8855 387.7592,234.7643 442.2047,266.1698 442.2047,267.053 387.3137,235.3816 387.1097,235.6642 262.8638,235.5218 238.9095,235.4637 238.9378,235.3302 200.5705,235.2887 118.1345,235.3089 118.0178,235.4842 118.0602,147.1898 117.8894,147.1004 117.8858,37.7953 117.1096,37.7953 117.1118,146.6929 117.2747,146.7783 117.2382,236.6548 117.0963,236.8679 238.6085,236.8839 238.584,236.9998 262.8638,237.0586 386.0089,237.1896 442.2047,269.7284 442.2047,268.8462 386.4051,236.6405 386.5549,236.4329 262.8638,236.2797 238.7492,236.2203 238.7741,236.1026 117.6161,236.0873"/>
 
 
 <polyline points="142.4375,198.8134 216.5868,198.8132 168.4743,115.7702 168.7023,114.5998 217.7149,199.4043 218.0882,199.5998 141.9153,199.5976 141.8827,37.7953 141.1073,37.7953 141.1882,200.6895 141.4055,200.3631 219.5631,200.3727 219.0084,200.082 168.93,113.4313 168.9916,37.7953 169.7639,37.7953 169.7813,109.0616 169.8409,108.756 224.2008,202.8026 223.9856,202.6898 139.8637,202.6785 139.6519,202.9965 139.5517,37.7953 138.7744,37.7953 138.7606,158.0359 138.9349,158.1272 138.8301,204.2305 226.9441,204.2399 170.2982,106.4087 170.069,107.5849 225.505,203.4859 139.3414,203.4628"/>
 
 
 <polyline points="209.9034,195.3114 167.3341,121

In [461]:
plot_id = fn.new_plot_id()

saved 20210919-222738_681190-4e22b-c04daf to s3://algorithmic-ink/current_plot_id


In [462]:
savedir='/home/naka/art/plotter_svgs'

In [463]:
savepath = Path(savedir).joinpath(f'{plot_id}.svg').as_posix()
sk.save(savepath)

In [386]:
sk.vpype('stat')

========= Stats ========= 
Current page size: (480.00000000000006, 672.0)
Layer 1
  Length: 235223.69629586654
  Pen-up length: 3783.959338661928
  Total length: 239007.65563452846
  Mean pen-up length: 20.127443290754936
  Median pen-up length: 8.334235320949919
  Path count: 189
  Segment count: 4776
  Mean segment length: 49.25119269176435
  Bounds: (37.795275590551185, 37.79527738494348, 442.2047244094489, 634.204722615056)
Totals
  Layer count: 1
  Length: 235223.69629586654
  Pen-up length: 3783.959338661928
  Total length: 239007.65563452846
  Path count: 189
  Segment count: 4776
  Mean segment length: 49.25119269176435
  Bounds: (37.795275590551185, 37.79527738494348, 442.2047244094489, 634.204722615056)


In [337]:
pmls.length

260761.31700853925